Installing all requirements(libraries)

In [1]:
!pip install unsloth
!pip install streamlit
!pip install streamlit cloudflared

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Granting access to the drive for runtime. You also need to provide the necessary permissions when requested by the pop-up.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Writing our code in app.py to run our model

In [3]:
%%writefile app.py
import streamlit as st
from unsloth import FastLanguageModel

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "drive/MyDrive/models/CET_Assist",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

# alpaca_prompt = You MUST copy from above!
# while True:
#   prompt = input("User: ")

#   if prompt.lower() == "exit":
#       break
st.title("Ask me anything about MHT-CET admissions!")

# Accept input from the user
prompt = st.text_input("prompt", value="")

# Tokenize input prompt
inputs = tokenizer(
[
    alpaca_prompt.format(
        "", # instruction
        prompt, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 864, use_cache = True)
decoded=tokenizer.batch_decode(outputs)
input_str = decoded[0]
response = input_str.split("### Response:")[1].split("<|eot_id|>")[0].strip()
# print("bot: "+response)
st.write(f"CET_Assist: {response}")

Writing app.py


Create a link for our Streamlit localhost; the local URL, network URL and external URL will not work. You need to use the link provided by Cloudflared(in the box).

In [ ]:
!streamlit run app.py & npx cloudflared tunnel --url http://localhost:8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.2:8501
  External URL: http://34.125.89.19:8501

2024-11-17T15:34:50Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2024-11-17T15:34:50Z INF Requesting new quick Tunnel on trycloudflare.com...
2024-11-17T15:34:54Z INF +--------------------------------------------------------------------------------------------+
2024-11-17T15:34:54Z INF | 